# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
1. Change to exercise08 repository

2. Start docker <br>
docker-compose up -d

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. copy the data to hdfs :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to hdfs needs to be done only once and it might take 1-2 minutes.)

## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: http://lars.yencken.org/datasets/languagegame/

## Preprocessing commands
In your newly created notebook run these commands in order to have the dataset into an RDD:

In [41]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
raw_data = sc.textFile(path)
dataset = raw_data.map(json.loads).cache()

After that you will be able to run the queries of the moodle question of this week. The RDD that you have to perform your queries on is the ```dataset``` one. For example, the following command returns one element of the dataset:

In [42]:
dataset.take(1)

[{'guess': 'Norwegian',
  'target': 'Norwegian',
  'country': 'AU',
  'choices': ['Maori', 'Mandarin', 'Norwegian', 'Tongan'],
  'sample': '48f9c924e0d98c959d8a6f1862b3ce9a',
  'date': '2013-08-19'}]

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [3]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

If you want to have a part of the dataset in order to run faster experiments then run the cell below and run your queries against the ```dataset_part``` RDD. However your final answer, the format of your final query and the productivity time should be run/measured against the whole dataset.

In [4]:
dataset_part = dataset.sample(False, 0.0001).cache()

## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Russian.

In [5]:
start_exercise()

In [43]:
# This exercise took 119s
# games_correct = dataset.filter(lambda game:game['guess']==game['target'])
# print(games_correct.take(5))
# print(games_correct.count())
dataset.filter(lambda game:game['guess']==game['target'] and game['target']=='Russian').count()

290818

In [8]:
finish_exercise()

This exercise took 119s


## Assignment 2
Return the number of distinct "target" languages.

In [13]:
start_exercise()

In [16]:
# This exercise took 170s
# target_langs = dataset.map(lambda game : game['target'])
# distinct_langs = target_langs.distinct()
# print(distinct_langs.take(5))
# print(distinct_langs.count())
dataset.map(lambda game : game['target']).distinct().count()

['Albanian', 'Slovenian', 'Tigrinya', 'Northern Ndebele', 'Japanese']
78


In [17]:
finish_exercise()

This exercise took 170s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [18]:
start_exercise()

In [26]:
# This exercise took 1928s
dataset.filter(lambda game:game['guess']==game['target']).sortBy(lambda game : (game['language'], game['country'], game['date']), ascending=True).map(game : game['sample']).take(3)

[{'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices': ['Albanian', 'Macedonian'],
  'sample': '00b85faa8b878a14f8781be334deb137',
  'date': '2013-09-04'},
 {'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices': ['Albanian', 'Bulgarian', 'Indonesian', 'Portuguese'],
  'sample': 'efcd813daec1c836d9f030b30caa07ce',
  'date': '2013-09-05'},
 {'guess': 'Albanian',
  'target': 'Albanian',
  'country': 'A1',
  'choices': ['Albanian', 'Hindi', 'Swahili'],
  'sample': '13722ceed1eede7ba597ade9b4cb9807',
  'date': '2013-09-08'}]

In [27]:
finish_exercise()

This exercise took 1928s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [28]:
start_exercise()

In [29]:
# This exercise took 744s
dataset.groupBy(lambda game : (game['country'], game['target'])).map(lambda x : (x[0], sum(1 for _ in x[1]))).sortBy(lambda y : y[1], ascending=False).take(3)

[(('US', 'French'), 112934),
 (('US', 'German'), 112007),
 (('US', 'Spanish'), 110919)]

In [30]:
finish_exercise()

This exercise took 744s


## Assignment 5
Find the percentage of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [31]:
start_exercise()

In [32]:
# This exercise took 235s
dataset.filter(lambda game:game['guess']==game['target'] and game['guess']==game['choices'][0]).count() / dataset.count()

0.25603983084476356

In [33]:
finish_exercise()

This exercise took 235s


## Assignment 6
Sort the languages by decreasing overall percentage of correct guesses and return the first three languages.

In [37]:
start_exercise()

In [39]:
# This exercise took 693s
dataset.map(lambda game : (game['target'] , 1 if game['target']==game['guess'] else 0)).groupByKey().map(lambda x : (x[0], sum(x[1])/sum(1 for _ in x[1]))).sortBy(lambda y:y[1], ascending=False).take(3)

[('French', 0.9382414927447232),
 ('German', 0.9197634593055483),
 ('Spanish', 0.8956432115670598)]

In [40]:
finish_exercise()

This exercise took 693s


## Assignment 7
Return the number games played on the latest day.

In [34]:
start_exercise()

In [35]:
# This exercise took 124s
latest_date = dataset.map(lambda game : game['date']).reduce(lambda x, y: max(x,y)); dataset.filter(lambda game: game['date']==latest_date).count()

65653

In [36]:
finish_exercise()

This exercise took 124s
